In [ ]:
import numpy as np
import pandas as pd
import os
import time
import random
import matplotlib.pyplot as plt 
import seaborn as sns

from glob import glob

import PIL
import cv2
import pickle

In [ ]:
from google.colab import files

uploaded = files.upload()

rgb2gray 된 traffic sign 이미지를 csv 형태로 가져왔다. (csv 받는 코드는 주피터를 통해서 가져왔다. 파일 업로드 시간 때문)

In [ ]:
train = pd.read_csv('simple_train.csv')
test = pd.read_csv('simple_test.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
train.iloc[:,[1024]]

In [ ]:
train_x = train.drop(['target'],axis = 1)
train_y = train.iloc[:,[1024]]
test_x = test.drop(['target'],axis = 1)
test_y = test.iloc[:,[1024]]

In [ ]:
x_train = train_x.to_numpy()
y_train = train_y.to_numpy()
x_test = test_x.to_numpy()
y_test = test_y.to_numpy()

In [ ]:
print(y_train[3675])
plt.imshow(x_train[3675].reshape(32,32))

In [ ]:
# 이 중에서 14(stop), 35(ahead only), 38(keep right)만 사용할 것이다.
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

모델링 1
: MNIST에서 사용한 DNN(로우레벨부터 개발, 튜닝X)을 바탕을 Naive Model로 설정하겠다. Gradient를 구하기가 용이해 FGSM 구현이 쉽다는 장점이 있다. 

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

class ANN:
    def __init__(self, layers_size):
        self.layers_size = layers_size
        self.parameters = {}
        self.L = len(self.layers_size)
        self.n = 0
        self.costs = []
    
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z))
        return expZ / expZ.sum(axis=0, keepdims=True)
    
    def initialize_parameters(self):
        for l in range(1, len(self.layers_size)):
            self.parameters["W" + str(l)] = np.random.randn(self.layers_size[l], self.layers_size[l - 1]) / np.sqrt(self.layers_size[l - 1])
            self.parameters["b" + str(l)] = np.zeros((self.layers_size[l], 1))
    
    def forward(self, X):
        store = {}
 
        A = X.T
        for l in range(self.L - 1):
            Z = self.parameters["W" + str(l + 1)].dot(A) + self.parameters["b" + str(l + 1)]
            A = self.sigmoid(Z)
            store["A" + str(l + 1)] = A
            store["W" + str(l + 1)] = self.parameters["W" + str(l + 1)]
            store["Z" + str(l + 1)] = Z
 
        Z = self.parameters["W" + str(self.L)].dot(A) + self.parameters["b" + str(self.L)]
        A = self.softmax(Z)
        store["A" + str(self.L)] = A
        store["W" + str(self.L)] = self.parameters["W" + str(self.L)]
        store["Z" + str(self.L)] = Z
 
        return A, store
 
    def sigmoid_derivative(self, Z):
        s = 1 / (1 + np.exp(-Z))
        return s * (1 - s)

    def backward(self,X,Y,store):

        derivatives = {}

        store['A0'] = X.T

        A = store['A' + str(self.L)]
        dZ = A - Y.T

        dW = dZ.dot(store['A' + str(self.L - 1)].T) / self.n
        db = np.sum(dZ, axis = 1, keepdims = True) / self.n
        dAPrev = store['W' + str(self.L)].T.dot(dZ)

        derivatives['dW' + str(self.L)] = dW
        derivatives['db' + str(self.L)] = db

        for l in range(self.L - 1,0,-1):
            dZ = dAPrev * self.sigmoid_derivative(store['Z' + str(l)])
            dW = dZ.dot(store['A' + str(l-1)].T) / self.n
            db = np.sum(dZ, axis = 1, keepdims = True) / self.n
            if l > 1:
                dAPrev = store['W' + str(l)].T.dot(dZ)
            derivatives["dW" + str(l)] = dW
            derivatives["db" + str(l)] = db

        return derivatives
    
    def backward_ad(self,X,Y,store):

        derivatives = {}

        store['A0'] = X.T

        A = store['A' + str(self.L)]
        dZ = A - Y.T

        dW = dZ.dot(store['A' + str(self.L - 1)].T) / self.n
        db = np.sum(dZ, axis = 1, keepdims = True) / self.n
        dAPrev = store['W' + str(self.L)].T.dot(dZ)

        derivatives['dW' + str(self.L)] = dW
        derivatives['db' + str(self.L)] = db

        for l in range(self.L - 1,0,-1):
            dZ = dAPrev * self.sigmoid_derivative(store['Z' + str(l)])
            dW = dZ.dot(store['A' + str(l-1)].T) / self.n
            db = np.sum(dZ, axis = 1, keepdims = True) / self.n
            if l > 1:
                dAPrev = store['W' + str(l)].T.dot(dZ)
            derivatives["dW" + str(l)] = dW
            derivatives["db" + str(l)] = db
            dAPrev_0 = store['W' + str(l)].T.dot(dZ)

        return dAPrev_0

    def fit(self, X, Y, learning_rate=0.01, n_iterations=300):
        self.n = X.shape[0]
 
        self.layers_size.insert(0, X.shape[1])
 
        self.initialize_parameters()
        for loop in range(n_iterations):
            A, store = self.forward(X)
            cost = -np.mean(Y * np.log(A.T+ 1e-8))
            derivatives = self.backward(X, Y, store)
 
            for l in range(1, self.L + 1):
                self.parameters["W" + str(l)] = self.parameters["W" + str(l)] - learning_rate * derivatives["dW" + str(l)]
                self.parameters["b" + str(l)] = self.parameters["b" + str(l)] - learning_rate * derivatives["db" + str(l)]
 
            if loop % 100 == 0:
                print("Cost: ", cost, "Train Accuracy:", self.predict(X, Y))
 
            if loop % 10 == 0:
                self.costs.append(cost)

    def predict(self, X, Y):
        A, cache = self.forward(X)
        y_hat = np.argmax(A, axis=0)
        Y = np.argmax(Y, axis=1)
        accuracy = (y_hat == Y).mean()
        return accuracy * 100
    
    def predict_output(self, X):
        A, cache = self.forward(X)
        y_hat = np.argmax(A, axis=0)
        return y_hat

    def plot_cost(self):
        plt.figure()
        plt.plot(np.arange(len(self.costs)), self.costs)
        plt.xlabel("epochs")
        plt.ylabel("cost")
        plt.show()

def pre_process_data(train_x, train_y, test_x, test_y):
    # Normalize
    train_x = train_x / 255.
    test_x = test_x / 255.
 
    enc = OneHotEncoder(sparse=False, categories='auto')
    train_y = enc.fit_transform(train_y.reshape(len(train_y), -1))
 
    test_y = enc.transform(test_y.reshape(len(test_y), -1))
    
    train_x_flatten = train_x.reshape(train_x.shape[0],-1)
    test_x_flatten = test_x.reshape(test_x.shape[0],-1)
    
    return train_x_flatten, train_y, test_x_flatten, test_y

if __name__ == '__main__':
    (train_x, train_y), (test_x, test_y) = (x_train, y_train), (x_test, y_test)
 
    train_x, train_y, test_x, test_y = pre_process_data(train_x, train_y, test_x, test_y)
 
    print("train_x's shape: " + str(train_x.shape))
    print("test_x's shape: " + str(test_x.shape))
    print("train_y's shape: " + str(train_y.shape))
    print("test_y's shape: " + str(test_y.shape))
 
 
    layers_dims = [50, 3]
 
    ann = ANN(layers_dims)
    ann.fit(train_x, train_y, learning_rate=0.1, n_iterations=3000)
    print("Train Accuracy:", ann.predict(train_x, train_y))
    print("Test Accuracy:", ann.predict(test_x, test_y))
    ann.plot_cost()    

In [ ]:
ann

FGSM 구현

In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
epsilon = 0.05

In [ ]:
# FGSM 공격 코드
def fgsm_attack(image, epsilon, data_grad):
    # data_grad 의 요소별 부호 값을 얻어옵니다
    sign_data_grad = np.sign(data_grad)
    perturbed_image = image + epsilon*sign_data_grad.T
    # 값 범위를 [0,1]로 유지하기 위해 자르기(clipping)를 추가합니다
    perturbed_image = np.clip(perturbed_image, 0, 1)
    # 작은 변화가 적용된 이미지를 리턴합니다
    return perturbed_image

In [ ]:
## predict_output에 한개씩 넣는 것부터 구현해야한다. (전체가 들어가는 것만 정상 작동함)

def test(model, test_x, test_y, epsilon, layers_dims):

    # 정확도 카운터
    correct = 0
    adv_examples = []

    # 테스트 셋의 모든 예제에 대해 루프를 돕니다
    
    for i in range(len(test_x)):
        data = test_x[i].reshape(1,1024)
        target = test_y[i].reshape(1,3)
        
        A,store = model.forward(data)
        output = model.predict_output(data)
        init_output = int(output)
        
        if init_output != int(np.argmax(target,axis = 1)):
            continue

        data_grad = model.backward_ad(data,target,store)
        
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        output = model.predict_output(perturbed_data)

        final_pred = int(output)

        if final_pred == int(np.argmax(target,axis = 1)):
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data
                adv_examples.append((init_output,final_pred,perturbed_data))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data
                adv_examples.append((init_output,final_pred,perturbed_data))

    # 해당 엡실론에서의 최종 정확도를 계산합니다
    final_acc = correct/float(len(test_x))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_x), final_acc))

    # 정확도와 적대적 예제를 리턴합니다
    return final_acc, adv_examples

In [ ]:
accuracies = []
examples = []

# 각 엡실론에 대해 테스트 함수를 실행합니다
for eps in epsilons:
    acc, ex = test(ann, test_x, test_y, eps, layers_dims)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# 각 엡실론에서 적대적 샘플의 몇 가지 예를 도식화합니다
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex.reshape((32,32)), cmap="gray")
plt.tight_layout()
plt.show()

In [ ]:
epsilons = [0, .005, .01, .015, .02, .025, .03]

In [ ]:
accuracies = []
examples = []

# 각 엡실론에 대해 테스트 함수를 실행합니다
for eps in epsilons:
    acc, ex = test(ann, test_x, test_y, eps, layers_dims)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .035, step=0.005))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")

In [ ]:
# 각 엡실론에서 적대적 샘플의 몇 가지 예를 도식화합니다
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex.reshape((32,32)), cmap="gray")
plt.tight_layout()
plt.show()

In [ ]:
epsilons = [0.1]

# 각 엡실론에서 적대적 샘플의 몇 가지 예를 도식화합니다
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex.reshape((32,32)), cmap="gray")
plt.tight_layout()
plt.show()